In [3]:
import pandas as pd
import geopandas as gpd

RNAL = pd.read_csv("../python/final/porto.csv")
RNAL["NrRNAL"] = RNAL["NrRNAL"].astype(str)
RNAL.head()

,Unnamed: 0,X,Y,OBJECTID,NrRNAL,Denominacao,DataRegisto,DataAberturaPublico,Modalidade,NrUtentes,Email,Endereco,CodigoPostal,Freguesia,Concelho,Distrito
0,100,-8.630320,41.158010,185476533,519,Koolhouse Porto,2014/12/05 10:19:59+00,2014/12/05 00:00:00+00,EstabelecimentoHospedagem,23,kendall.faria@gmail.com,Avenida Avenida Boavista 911,4100-128,"União das freguesias de Cedofeita, Santo Ildef...",Porto,Porto
1,293,-8.602860,41.144624,185476726,14,Centenry Fontaínhas Apartments 1,2014/12/01 09:40:52+00,2014/12/01 00:00:00+00,Apartamento,6,apcf.pc@gmail.com,Rua Fontaínhas 167 1.º,4000-240,"União das freguesias de Cedofeita, Santo Ildef...",Porto,Porto
2,294,-8.602860,41.144624,185476727,15,Centenry Fontaínhas Apartments 1,2014/12/01 09:56:17+00,2014/12/01 00:00:00+00,Apartamento,6,apcf.pc@gmail.com,Rua Fontaínhas 167,4000-240,"União das freguesias de Cedofeita, Santo Ildef...",Porto,Porto
3,295,-8.602860,41.144624,185476728,16,Centenry Fontaínhas Apartments 3,2014/12/01 10:04:00+00,2014/12/01 00:00:00+00,Apartamento,6,apcf.pc@gmail.com,Rua Fontaínhas 167 3.º,4000-240,"União das freguesias de Cedofeita, Santo Ildef...",Porto,Porto
4,296,-8.614323,41.142069,185476729,32,inPátio Guest House,2014/12/01 16:41:53+00,2012/09/10 00:00:00+00,EstabelecimentoHospedagem,12,info@inpatio.pt,Pátio São Salvador 22,4050-567,"União das freguesias de Cedofeita, Santo Ildef...",Porto,Porto


In [5]:
input_file = "BGRI/BGRI.gpkg"
BGRI = gpd.read_file(input_file)

BGRI.head()

,OBJECTID,BGRI2021,DT21,DTMN21,DTMNFR21,DTMNFRSEC21,SECNUM21,SSNUM21,SECSSNUM21,SUBSECCAO,...,N_INDIVIDUOS,N_INDIVIDUOS_H,N_INDIVIDUOS_M,N_INDIVIDUOS_0_14,N_INDIVIDUOS_15_24,N_INDIVIDUOS_25_64,N_INDIVIDUOS_65_OU_MAIS,SHAPE_Length,SHAPE_Area,geometry
0,52,13121000702,13,1312,131210,131210007,007,02,00702,13121000702,...,328.0,145.0,183.0,29.0,34.0,184.0,81.0,612.308514,21064.473451,"MULTIPOLYGON (((-40792.851 166989.435, -40733...."
1,53,13120302118,13,1312,131203,131203021,021,18,02118,13120302118,...,134.0,54.0,80.0,8.0,10.0,55.0,61.0,481.233389,13437.276410,"MULTIPOLYGON (((-37680.254 167001.264, -37667...."
2,54,13121103402,13,1312,131211,131211034,034,02,03402,13121103402,...,143.0,67.0,76.0,15.0,8.0,79.0,41.0,707.178859,13484.048739,"MULTIPOLYGON (((-43406.159 166894.230, -43347...."
3,55,13121600903,13,1312,131216,131216009,009,03,00903,13121600903,...,205.0,89.0,116.0,33.0,12.0,100.0,60.0,824.199641,24498.620921,"MULTIPOLYGON (((-45059.181 166999.619, -45056...."
4,56,13121600403,13,1312,131216,131216004,004,03,00403,13121600403,...,5.0,2.0,3.0,0.0,1.0,2.0,2.0,177.035284,1829.033862,"MULTIPOLYGON (((-43954.343 166993.913, -43965...."


In [16]:
BGRI = BGRI.to_crs("EPSG:4326")
points_gdf = gpd.GeoDataFrame(RNAL, geometry=gpd.points_from_xy(RNAL['X'], RNAL['Y']), crs="EPSG:4326")
points_in_areas = gpd.sjoin(points_gdf, BGRI, how='left', op='within')

# Count the points in each area
point_counts = points_in_areas.groupby('BGRI2021').size().reset_index(name='point_count')


        BGRI2021  point_count
0    13120200101           21
1    13120200104           28
2    13120200105            1
3    13120200106            3
4    13120200107           34
..           ...          ...
809  13121802505            1
810  13121802506            5
811  13121802510            4
812  13121802511            8
813  13121802512            1

[814 rows x 2 columns]


/opt/homebrew/opt/ipython/libexec/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [19]:
point_counts["point_count"].value_counts()

point_count
1     209
2     108
3      67
4      46
5      43
     ... 
41      1
57      1
74      1
93      1
50      1
Name: count, Length: 84, dtype: int64

In [20]:
merged_df = pd.merge(BGRI, point_counts, on='BGRI2021', how='left')

# Fill NaN values with 0 in the 'number_of_inhabitants' column
merged_df['point_count'] = merged_df['point_count'].fillna(0)

In [23]:
merged_df = merged_df.rename(columns={"point_count":"ALs"})

In [26]:
merged_df["pressao"] = merged_df["ALs"] / merged_df["N_ALOJAMENTOS_FAM_CLASS_RHABITUAL"]

In [31]:
merged_df["N_ALOJAMENTOS_FAMILIARES"].value_counts()

N_ALOJAMENTOS_FAMILIARES
0.0      55
12.0     30
16.0     29
10.0     27
24.0     26
         ..
239.0     1
282.0     1
225.0     1
162.0     1
235.0     1
Name: count, Length: 296, dtype: int64